### Partie 4 : Environnement FrozenLake

Voir https://gymnasium.farama.org/environments/toy_text/frozen_lake/

In [1]:
import gymnasium
from gymnasium.envs.toy_text.frozen_lake import generate_random_map


class FrozenLakeEnv(gym.Env) :
    def __init__(self, lake_map = generate_random_map(size=4)) :
        super(FrozenLakeEnv, self).__init__()
        self.size = len(lake_map)
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Discrete(size ** 2)
        self.state = 0
        self.map = lake_map
    
    def reset(self) :
        self.state = 0
        return self.state
    
    def step(self, action) :
        x, y = self._stateToCoordinate(self.state)
        if action == 0 : # Left
            if x > 0 :
                x -= 1
        elif action == 1 : # Down
            if y < self.size -1 :
                y += 1
        elif action == 2 : # Right
            if x < self.size -1 :
                x += 1
        elif action == 3 : # Up
            if y > 0 :
                y -= 1
        
        self.state = self._coordinateToState(x, y)
        
        result = self.map[y][x]
        done = result in ['H', 'G']
        if result == 'G' :
            reward = 1
        elif result == 'H' :
            reward = 0
        else :
            reward = 0
        return self.state, reward, done, {}
    
    
    def _stateToCoordinate(self, state) :
        x = state % self.size
        y = state // self.size
        return x, y
    
    def _coordinateToState(self, x, y) :
        return self.size*y + x
    
    def render(self, mode="human") :
        x, y = self._stateToCoordinate(self.state)
        toPrint = ''
        for i in range(self.size) :
            toPrintRow = ''
            row = self.map[i]
            for j in range(self.size) :
                if (j, i) == (x, y):
                    toPrintRow += '.'
                else :
                    toPrintRow += row[j]
            toPrint += toPrintRow + '\n'
        print(toPrint)
        
    def seed(self, seedInt):
        pass
        #determinist, needed for stablebaselines3

NameError: name 'gym' is not defined

## Observations

### 2. Observation FrozeLake non slippery

In [ ]:
import matplotlib.pyplot as plt

data = [(4, 0, 0), (6, 0, 1), (8, 1, 0), (10, 1, 1)]
fig, axes = plt.subplots(2, 2, figsize=(15, 15))


for (size, x, y) in data :
    lake_map = generate_random_map(size = size)
    env = FrozenLakeEnv(lake_map = lake_map)
    env.render()
    DQNmodel, DQNrewards = train_dqn(env, lr = 0.01, buffer_size = 500, 
              exploration_fraction = 0.3, exploration_final_eps = 0.05, 
              timesteps = 10000)
    QTable, QRewards = train_q_learning(env, episodes = len(DQNrewards),  alpha = 0.1, gamma = 0.9, epsilon = 0.7)
    
    
    axes[x, y].plot([r for r in QRewards], label='QLearning')
    axes[x, y].plot([r for r in DQNrewards], label='DQN')
    axes[x, y].set_xlim([0, 200])
    axes[x, y].set_ylim([-1, 2])
    axes[x, y].legend()
    axes[x, y].set_xlabel("Épisode")
    axes[x, y].set_ylabel("Récompense")
    axes[x, y].set_title(f"Récompense par épisode pour carte de taille {size}")
    axes[x, y].grid()
plt.show()